In [ ]:
# Install dependencies with compatible torchvision version
!pip install transformers==4.41.2 torch==2.3.0 torchvision==0.18.0 cryptography==42.0.8 syft==0.9.1 matplotlib==3.8.4 pandas==2.2.2 numpy==1.26.4 scikit-learn==1.5.0 kaggle==1.6.17 huggingface_hub==0.25.2 imbalanced-learn==0.12.3 --force-reinstall

# Setup Kaggle API (upload kaggle.json)
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download datasets
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!kaggle datasets download -d mathchi/diabetes-data-set
!unzip chest-xray-pneumonia.zip -d chest_xray
!unzip diabetes-data-set.zip -d diabetes

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.3.0-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.18.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached syft-0.9.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached matplotlib-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached kaggle-1.6.17.tar.gz (82 kB)
  Preparing metadata (setup.py) ... done
  Using cached huggingface_hub-0.25.2-py3-none-any.whl.metadata (13 kB)
  Using cached imbalanced_lear

Streaming output truncated to the last 5000 lines.
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0437-0001-0002.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0437-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0438-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0439-0001-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0439-0001-0002.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0439-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0440-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0441-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0442-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0444-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0445-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0446-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM-0447-0001.jpeg  
  inflating: chest_xray/chest_xray/train/NORMAL/IM

In [ ]:
# Install dependencies with compatible torchvision version
!pip install transformers==4.41.2 torch==2.3.0 torchvision==0.18.0 cryptography==42.0.8 syft==0.9.1 matplotlib==3.8.4 pandas==2.2.2 numpy==1.26.4 scikit-learn==1.5.0 kaggle==1.6.17 huggingface_hub==0.25.2 imbalanced-learn==0.12.3 --force-reinstall
!pip install --upgrade syft # upgrade syft
!pip install --force-reinstall syft==0.9.1 # reinstall syft
# Import libraries
import pandas as pd
import numpy as np
import random
import torch
import syft as sy
from transformers import BertTokenizer, BertForSequenceClassification
from cryptography.fernet import Fernet
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import os
from torch.utils.data import TensorDataset, DataLoader  # Added this line

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.3.0-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.18.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached syft-0.9.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached matplotlib-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached kaggle-1.6.17.tar.gz (82 kB)
  Preparing metadata (setup.py) ... done
  Using cached huggingface_hub-0.25.2-py3-none-any.whl.metadata (13 kB)
  Using cached imbalanced_lear

  Using cached syft-0.9.1-py2.py3-none-any.whl.metadata (16 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_ro

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.ChunkedArray size changed, may indicate binary incompatibility. Expected 64 from C header, got 72 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib._Tabular size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Table size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
syft 0.9.5 requires numpy<=1.24.4,>=1.23.5; python_version < "3.12", but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [ ]:
!pip uninstall -y numpy jax tensorflow
!pip install numpy==1.25.0 jax==0.4.14
!pip install transformers==4.41.2 torch==2.3.0 torchvision==0.18.0 cryptography==42.0.8 syft==0.9.1 matplotlib==3.8.4 pandas==2.2.2 scikit-learn==1.5.0 kaggle==1.6.17 huggingface_hub==0.25.2 imbalanced-learn==0.12.3 --force-reinstall

Found existing installation: numpy 1.25.0
Uninstalling numpy-1.25.0:
  Successfully uninstalled numpy-1.25.0
Found existing installation: jax 0.4.14
Uninstalling jax-0.4.14:
  Successfully uninstalled jax-0.4.14
  Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached jax-0.4.14-py3-none-any.whl
Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, which is not installed.
syft 0.9.5 requires numpy<=1.24.4,>=1.23.5; python_version < "3.12", but you have numpy 1.25.0 which is incompatible.
flax 0.10.6 requires jax>=0.5.1, but you have jax 0.4.14 which is incompatible.
thinc 8.3.6 re

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.3.0-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.18.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
ERROR: Operation cancelled by user
^C


In [ ]:
# Load and preprocess the diabetes dataset
!pip install --upgrade jax jaxlib # Upgrade both jax and jaxlib to the latest compatible versions
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
import random
import torch
import syft as sy
from transformers import BertTokenizer, BertForSequenceClassification
from cryptography.fernet import Fernet
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import os
from torch.utils.data import TensorDataset, DataLoader
df = pd.read_csv('diabetes/diabetes.csv')
X = df.drop(columns=['Outcome'])
y = df['Outcome']

# Handle zeros in Insulin and SkinThickness
for col in ['Insulin', 'SkinThickness']:
    X[col] = X[col].replace(0, X[col].median())

# Stratified split for two simulated clients on the original dataset
from sklearn.model_selection import train_test_split
client1_data, client2_data = train_test_split(df, test_size=0.5, stratify=df['Outcome'], random_state=42)

client1_X = client1_data.drop(columns=['Outcome'])
client1_y = client1_data['Outcome']
client2_X = client2_data.drop(columns=['Outcome'])
client2_y = client2_data['Outcome']

# Apply SMOTETomek for balancing on each client's data
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(sampling_strategy=0.67, random_state=42)
client1_X_balanced, client1_y_balanced = smote_tomek.fit_resample(client1_X, client1_y)
client2_X_balanced, client2_y_balanced = smote_tomek.fit_resample(client2_X, client2_y)

# Handle zeros in the balanced datasets
for col in ['Insulin', 'SkinThickness']:
    client1_X_balanced[col] = client1_X_balanced[col].replace(0, client1_X_balanced[col].median())
    client2_X_balanced[col] = client2_X_balanced[col].replace(0, client2_X_balanced[col].median())

# Split each client's balanced data into train and validation sets
client1_X_train, client1_X_val, client1_y_train, client1_y_val = train_test_split(client1_X_balanced, client1_y_balanced, test_size=0.2, stratify=client1_y_balanced, random_state=42)
client2_X_train, client2_X_val, client2_y_train, client2_y_val = train_test_split(client2_X_balanced, client2_y_balanced, test_size=0.2, stratify=client2_y_balanced, random_state=42)

# Print class distribution for each client
client1_df_balanced = pd.DataFrame(client1_X_train, columns=X.columns)
client1_df_balanced['Outcome'] = client1_y_train
client2_df_balanced = pd.DataFrame(client2_X_train, columns=X.columns)
client2_df_balanced['Outcome'] = client2_y_train
print("Class distribution in Outcome (Client 1):")
print(client1_df_balanced['Outcome'].value_counts(normalize=True))
print("Class distribution in Outcome (Client 2):")
print(client2_df_balanced['Outcome'].value_counts(normalize=True))

# Import XGBoost
from xgboost import XGBClassifier
import tempfile
import os
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, precision_recall_curve
from cryptography.fernet import Fernet

# Generate encryption key
key = Fernet.generate_key()
cipher = Fernet(key)

# Simulated federated training loop
num_rounds = 50
client_data = [(client1_X_train, client1_y_train, client1_X_val, client1_y_val), (client2_X_train, client2_y_train, client2_X_val, client2_y_val)]

# Calculate scale_pos_weight based on original dataset
scale_pos_weight = (y == 0).sum() / (y == 1).sum()

# Hyperparameter tuning with GridSearchCV on original dataset
param_grid = {
    'n_estimators': [20, 50],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.3],
    'reg_lambda': [1, 5],
    'reg_alpha': [0.5, 1]
}
grid_search = GridSearchCV(XGBClassifier(eval_metric='logloss', random_state=42, scale_pos_weight=scale_pos_weight),
                           param_grid, cv=5, scoring='f1')
grid_search.fit(X, y)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated F1:", grid_search.best_score_)

# Use the best model
global_model = grid_search.best_estimator_

# Simulated federated training loop with weighted aggregation
client_predictions = []
client_weights = []

for round in range(3):  # Simulate 3 rounds of federated training
    round_client_models = []
    round_client_weights = []

    for client_idx, (X_train_client, y_train_client, X_val_client, y_val_client) in enumerate(client_data):
        # Create a fresh model with tuned parameters and slight variation
        client_model = XGBClassifier(**grid_search.best_params_, eval_metric='logloss', random_state=42 + client_idx, scale_pos_weight=scale_pos_weight)

        # Train on client training data
        client_model.fit(X_train_client, y_train_client)

        # Evaluate on client validation data to compute weight
        val_preds = client_model.predict(X_val_client)
        val_f1 = f1_score(y_val_client, val_preds)
        round_client_weights.append(val_f1)

        # Save model to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix='.json') as tmp:
            client_model.save_model(tmp.name)
            with open(tmp.name, 'rb') as f:
                model_bytes = f.read()
            encrypted_model = cipher.encrypt(model_bytes)
            round_client_models.append(encrypted_model)
        os.unlink(tmp.name)

    # Collect prediction probabilities from each client
    predictions_per_client = []
    for client_idx, (X_train_client, _, _, _) in enumerate(client_data):
        decrypted_model_bytes = cipher.decrypt(round_client_models[client_idx])
        with tempfile.NamedTemporaryFile(delete=False, suffix='.json') as tmp:
            with open(tmp.name, 'wb') as f:
                f.write(decrypted_model_bytes)
            client_model = XGBClassifier(**grid_search.best_params_, eval_metric='logloss', random_state=42 + client_idx, scale_pos_weight=scale_pos_weight)
            client_model.load_model(tmp.name)
        os.unlink(tmp.name)

        # Get prediction probabilities on the entire dataset
        preds = client_model.predict_proba(X)
        predictions_per_client.append(preds)

    # Store predictions and weights for ensemble
    client_predictions.append(predictions_per_client)
    client_weights.append(round_client_weights)
    print(f'Round {round+1}/3 completed')

# Weighted ensemble aggregation (improved)
client_predictions = np.array(client_predictions)  # Shape: (n_rounds, n_clients, n_samples, n_classes)
client_weights = np.array(client_weights)  # Shape: (n_rounds, n_clients)

# Normalize weights within each round
client_weights = client_weights / client_weights.sum(axis=1, keepdims=True)

# Weighted average per round
round_predictions = []
for round_idx in range(client_predictions.shape[0]):
    weighted_round_preds = np.average(client_predictions[round_idx], axis=0, weights=client_weights[round_idx])
    round_predictions.append(weighted_round_preds)

# Average across rounds
weighted_predictions = np.mean(round_predictions, axis=0)  # Shape: (n_samples, n_classes)

# Optimize classification threshold
# Use a small validation set from the original data to find the best threshold
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
val_probs = global_model.predict_proba(X_val)[:, 1]  # Probabilities for class 1
precisions, recalls, thresholds = precision_recall_curve(y_val, val_probs)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
best_threshold = thresholds[np.argmax(f1_scores)]
print(f"Best threshold for F1 score: {best_threshold:.4f}")

# Apply the best threshold
final_predictions = (weighted_predictions[:, 1] >= best_threshold).astype(int)

# Evaluate the ensemble predictions
print("Unique predictions:", np.unique(final_predictions))
accuracy = accuracy_score(y, final_predictions)
f1 = f1_score(y, final_predictions)
print(f'Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')

# Cross-validation with the tuned model
scores = cross_val_score(global_model, X, y, cv=5, scoring='f1')
print(f'Cross-validated F1 Score: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})')

# Plot results with display fix
plt.figure(figsize=(8, 6))
plt.bar(['Accuracy', 'F1 Score'], [accuracy, f1], color=['blue', 'green'])
plt.title('Model Performance (XGBoost)')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.savefig('performance_xgboost.png')
plt.close()
from IPython.display import Image
display(Image('performance_xgboost.png'))

# Feature importance plot
importances = global_model.feature_importances_
feature_names = X.columns  # Use original feature set (8 features)
plt.figure(figsize=(8, 6))
plt.barh(feature_names, importances, color='purple')
plt.title('Feature Importance (XGBoost)')
plt.xlabel('Importance')
plt.savefig('feature_importance.png')
plt.close()
display(Image('feature_importance.png'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 89.3 MB/s eta 0:00:00
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.5.1
    Uninstalling jaxlib-0.5.1:
      Successfully uninstalled jaxlib-0.5.1
  Attempting uninstall: jax
    Found existing installation: jax 0.4.14
    Uninstalling jax-0.4.14:
      Successfully uninstalled jax-0.4.14
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)

# Ensure all dependencies are installed

In [ ]:
!pip install --upgrade numpy

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
syft 0.9.1 requires numpy<=1.24.4,>=1.23.5; python_version < "3.12", but you have numpy 2.2.5 which is incompatible.
pyarrow 15.0.0 requires numpy<2,>=1.16.6, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 whic

In [ ]:
!pip install jax --upgrade

In [ ]:
!pip install transformers==4.41.2 torch==2.3.0 torchvision==0.18.0 cryptography==42.0.8 syft==0.9.1 matplotlib==3.8.4 pandas==2.2.2 numpy==1.26.4 scikit-learn==1.5.0 kaggle==1.6.17 huggingface_hub==0.25.2 imbalanced-learn==0.12.3

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.3.0-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.18.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached cryptography-42.0.8-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached matplotlib-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached kaggle-1.6.17.tar.gz (82 kB)
  Preparing metadata (setup.py) ... done
  Using cached huggingface_hub-0.25.2-py3-none-any.whl.metadata (13 kB)
  Using cached imbalanced_learn-0.12.3-py3-none-any.whl.metadata (8.3 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached

In [ ]:
!pip install numpy==1.25.0

  Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
syft 0.9.1 requires numpy<=1.24.4,>=1.23.5; python_version < "3.12", but you have numpy 1.25.0 which is incompatible.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.0 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.25.0 which

In [ ]:
!pip install importlib-metadata --upgrade
!pip install torch --upgrade

# Import libraries
import pandas as pd
import numpy as np
import random
import torch
import syft as sy # Now this line should work without error
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import tempfile
import os
import time
import matplotlib.pyplot as plt
import logging

# Set random seed for reproducibility

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# No SEAL import due to installation failure; using simulated encryption
SEAL_AVAILABLE = False
logger.warning("Microsoft SEAL library not installed. Using simulated encryption. To implement full homomorphic encryption, set up a local environment with Python 3.8 and install pysealc.")

# Set random seed for reproducibility
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Generate Simulated Radiology Reports for Chest X-Ray Dataset

In [ ]:
# 1. Generate Simulated Radiology Reports for Chest X-Ray Dataset
def generate_radiology_reports(labels, num_samples=5856):
    """Generate simulated radiology reports for Chest X-Ray dataset."""
    reports = []
    for i in range(num_samples):
        label = labels[i % len(labels)]
        if label == 0:  # Normal
            report = "Chest X-ray shows clear lungs with no signs of pneumonia or abnormalities."
        else:  # Pneumonia
            report = "Chest X-ray reveals bilateral infiltrates consistent with pneumonia."
        reports.append(report)
    return reports

# Load Chest X-Ray labels (assuming binary labels are available)
# For simplicity, simulate labels (replace with actual labels if available)
chest_xray_labels = np.random.randint(0, 2, size=5856)  # 5856 images
radiology_reports = generate_radiology_reports(chest_xray_labels)






# Generate Simulated Clinical Notes for Diabetes Dataset

In [ ]:
# 2. Generate Simulated Clinical Notes for Diabetes Dataset
def generate_diabetes_notes(labels, num_samples=768):
    """Generate simulated clinical notes for Diabetes Dataset."""
    notes = []
    for i in range(num_samples):
        label = labels[i % len(labels)]
        if label == 0:  # No diabetes
            note = "Patient has normal glucose levels and BMI, no signs of diabetes."
        else:  # Diabetes
            note = "Patient exhibits high glucose levels and elevated BMI, diagnosed with diabetes."
        notes.append(note)
    return notes

# Load Diabetes labels (assuming binary labels are available)
# For simplicity, simulate labels (replace with actual labels if available)
diabetes_labels = np.random.randint(0, 2, size=768)  # 768 records
diabetes_notes = generate_diabetes_notes(diabetes_labels)

# Generate Synthetic Genomic Data

In [ ]:
!pip uninstall -y tokenizers

Found existing installation: tokenizers 0.13.3
Uninstalling tokenizers-0.13.3:
  Successfully uninstalled tokenizers-0.13.3


In [ ]:
!pip install tokenizers==0.13.3

  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.28.1 which is incompatible.


In [ ]:
!pip install transformers==4.31.0

  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.1
    Uninstalling transformers-4.28.1:
      Successfully uninstalled transformers-4.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
print("Transformers imported successfully!")

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.6.0, so it will not be used.
  warnings.warn(


Transformers imported successfully!


In [ ]:
import random  # Import the random module

# 3. Generate Synthetic Genomic Data
def generate_synthetic_genomic_data(num_sequences=500, seq_length=100):
    """Generate synthetic genomic sequences with mutation labels."""
    bases = ['A', 'T', 'C', 'G']
    sequences = []
    labels = []
    for _ in range(num_sequences):
        sequence = ''.join(random.choice(bases) for _ in range(seq_length))
        # Simulate mutations (e.g., 10% chance of mutation)
        if random.random() < 0.1:
            label = 1  # Mutation
            # Introduce a random mutation
            pos = random.randint(0, seq_length-1)
            sequence = sequence[:pos] + random.choice(bases) + sequence[pos+1:]
        else:
            label = 0  # No mutation
        sequences.append(sequence)
        labels.append(label)
    return sequences, labels

import logging # Import logging module

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__) # Create logger instance

try:
    genomic_sequences, genomic_labels = generate_synthetic_genomic_data()
    genomic_texts = [' '.join(seq) for seq in genomic_sequences]
    logger.info("Synthetic genomic data generated successfully.")
except Exception as e:
    logger.error(f"Error generating genomic data: {e}")
    raise

# Fine-Tune DistilBERT for Text Classification

In [ ]:
# 4. Fine-Tune DistilBERT for Text Classification
import torch # Import torch here
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


def train_distilbert(texts, labels, dataset_name):
    try:
        # Split data
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            texts, labels, test_size=0.2, random_state=42
        )
        logger.info(f"Data split completed for {dataset_name}.")

        # Initialize tokenizer and model
        tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
        logger.info(f"DistilBERT loaded for {dataset_name}.")

        # Create datasets
        train_dataset = TextDataset(train_texts, train_labels, tokenizer)
        val_dataset = TextDataset(val_texts, val_labels, tokenizer)

        # Define training arguments
        training_args = TrainingArguments(
            output_dir=f'./results_{dataset_name}',
            num_train_epochs=1,  # Reduced for faster testing
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir=f'./logs_{dataset_name}',
            logging_steps=10
        )

        # Define compute metrics
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            acc = accuracy_score(labels, preds)
            f1 = f1_score(labels, preds)
            return {'accuracy': acc, 'f1': f1}

        # Initialize trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        # Train and evaluate
        trainer.train()
        results = trainer.evaluate()
        logger.info(f"{dataset_name} Evaluation Results: {results}")
        trainer.save_model(f'./best_model_{dataset_name}')
        return model, tokenizer, val_texts, val_labels
    except Exception as e:
        logger.error(f"Error training DistilBERT on {dataset_name}: {e}")
        raise

# Train on all datasets
try:
    chest_model, chest_tokenizer, chest_val_texts, chest_val_labels = train_distilbert(
        radiology_reports, chest_xray_labels, "Chest_XRay"
    )
    diabetes_model, diabetes_tokenizer, diabetes_val_texts, diabetes_val_labels = train_distilbert(
        diabetes_notes, diabetes_labels, "Diabetes"
    )
    genomic_model, genomic_tokenizer, genomic_val_texts, genomic_val_labels = train_distilbert(
        genomic_texts, genomic_labels, "Genomic"
    )
except Exception as e:
    logger.error(f"Error training models: {e}")
    raise

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


...

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['embeddings.word_embeddings.weight', 'encoder.layer.7.attention.output.dense.weight', 'encoder.layer.2.attention.self.qu

Step,Training Loss
10,0.695100
20,0.678300
30,0.631900
40,0.584400
50,0.453900
60,0.279600
70,0.065900


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['embeddings.word_embeddings.weight', 'encoder.layer.7.attention.output.dense.weight', 'encoder.layer.2.attention.self.qu

Step,Training Loss
10,0.566800
20,0.382000
30,0.273500
40,0.454500
50,0.281000


# Simulated Encryption Functions

In [ ]:
# 5. Simulated Encryption Functions
def simulate_encrypt_weights(weights):
    """Simulate encryption by adding noise proportional to weight magnitude."""
    return [w + np.random.normal(0, 0.01 * np.abs(w).mean(), w.shape) for w in weights]

def simulate_decrypt_weights(encrypted_weights):
    """Simulate decryption by returning the weights."""
    return encrypted_weights

# Federated Learning with DistilBERT and Simulated Encryption

In [ ]:
6. Federated Learning with DistilBERT and Simulated Encryption
# Simulate client data splits for Chest X-Ray (3 clients)
num_clients = 3
chest_client_data = []
train_reports, _, train_labels, _ = train_test_split(
    radiology_reports, chest_xray_labels, test_size=0.2, random_state=42
)
for i in range(num_clients):
    start_idx = i * len(train_reports) // num_clients
    end_idx = (i + 1) * len(train_reports) // num_clients
    client_reports = train_reports[start_idx:end_idx]
    client_labels = train_labels[start_idx:end_idx]
    client_val_texts = chest_val_texts[i * len(chest_val_texts) // num_clients:(i + 1) * len(chest_val_texts) // num_clients]
    client_val_labels = chest_val_labels[i * len(chest_val_labels) // num_clients:(i + 1) * len(chest_val_labels) // num_clients]
    chest_client_data.append((client_reports, client_labels, client_val_texts, client_val_labels))

# Federated training loop for Chest X-Ray
global_model = chest_model
client_predictions = []
client_weights = []
encryption_times = []
decryption_times = []

for round in range(3):  # 3 rounds of federated training
    round_client_models = []
    round_client_weights = []
    for client_idx, (client_reports, client_labels, client_val_texts, client_val_labels) in enumerate(chest_client_data):
        try:
            # Create client dataset
            client_train_dataset = TextDataset(client_reports, client_labels, chest_tokenizer)
            client_val_dataset = TextDataset(client_val_texts, client_val_labels, chest_tokenizer)

            # Initialize client model
            client_model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
            client_trainer = Trainer(
                model=client_model,
                args=TrainingArguments(
                    output_dir=f'./results_client_{client_idx}',
                    num_train_epochs=1,
                    per_device_train_batch_size=8,
                    per_device_eval_batch_size=8,
                    warmup_steps=500,
                    weight_decay=0.01,
                    logging_dir=f'./logs_client_{client_idx}',
                    logging_steps=10
                ),
                train_dataset=client_train_dataset,
                eval_dataset=client_val_dataset,
                compute_metrics=compute_metrics
            )

            # Train client model
            client_trainer.train()

            # Evaluate client model
            val_results = client_trainer.evaluate()
            val_f1 = val_results['eval_f1']
            round_client_weights.append(val_f1)

            # Simulate encryption
            start_time = time.time()
            model_weights = [param.data.cpu().numpy().flatten() for param in client_model.parameters()]
            encrypted_weights = simulate_encrypt_weights(model_weights)
            encryption_times.append(time.time() - start_time)

            round_client_models.append(encrypted_weights)
        except Exception as e:
            logger.error(f"Error in client {client_idx} training: {e}")
            raise

    # Aggregate encrypted weights
    try:
        aggregated_weights = []
        for layer_idx in range(len(round_client_models[0])):
            aggregated_layer = []
            for neuron_idx in range(len(round_client_models[0][layer_idx])):
                weighted_sum = np.zeros_like(round_client_models[0][layer_idx][neuron_idx])
                for client_idx in range(num_clients):
                    client_weight = round_client_weights[client_idx] / sum(round_client_weights)
                    weighted_sum += client_weight * round_client_models[client_idx][layer_idx][neuron_idx]
                aggregated_layer.append(weighted_sum)
            aggregated_weights.append(aggregated_layer)

        # Simulate decryption
        start_time = time.time()
        decrypted_weights = simulate_decrypt_weights(aggregated_weights)
        decryption_times.append(time.time() - start_time)

        # Update global model
        with torch.no_grad():
            for param, dec_w in zip(global_model.parameters(), decrypted_weights):
                param.copy_(torch.tensor(dec_w).reshape(param.shape))
    except Exception as e:
        logger.error(f"Error in weight aggregation: {e}")
        raise

    # Collect predictions
    try:
        predictions_per_client = []
        for client_idx, (client_reports, _, client_val_texts, client_val_labels) in enumerate(chest_client_data):
            client_val_dataset = TextDataset(client_val_texts, client_val_labels, chest_tokenizer)
            preds = client_trainer.predict(client_val_dataset).predictions.argmax(-1)
            predictions_per_client.append(preds)
        client_predictions.append(predictions_per_client)
        client_weights.append(round_client_weights)
        logger.info(f'Round {round+1}/3 completed')
    except Exception as e:
        logger.error(f"Error in prediction collection: {e}")
        raise

# Output Metrics and Visualize Results

In [ ]:
# 7. Output Metrics and Visualize Results
# Calculate final metrics for Chest X-Ray
try:
    final_predictions = np.concatenate([np.concatenate(round_preds, axis=0) for round_preds in client_predictions], axis=0)
    final_labels = np.concatenate([client_val_labels for _, _, _, client_val_labels in chest_client_data])
    final_accuracy = accuracy_score(final_labels[:len(final_predictions)], final_predictions)
    final_f1 = f1_score(final_labels[:len(final_predictions)], final_predictions)
    avg_encryption_time = np.mean(encryption_times) if encryption_times else 0
    avg_decryption_time = np.mean(decryption_times) if decryption_times else 0

    logger.info(f"Chest X-Ray Final Federated Accuracy: {final_accuracy:.4f}")
    logger.info(f"Chest X-Ray Final Federated F1 Score: {final_f1:.4f}")
    logger.info(f"Average Encryption Time: {avg_encryption_time:.4f} seconds")
    logger.info(f"Average Decryption Time: {avg_decryption_time:.4f} seconds")
except Exception as e:
    logger.error(f"Error calculating Chest X-Ray metrics: {e}")
    raise

# Plot metrics
try:
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.bar(['Accuracy', 'F1 Score'], [final_accuracy, final_f1])
    plt.title('Federated Learning Metrics (Chest X-Ray)')
    plt.subplot(1, 2, 2)
    plt.bar(['Encryption Time', 'Decryption Time'], [avg_encryption_time, avg_decryption_time])
    plt.title('Computational Overhead')
    plt.savefig('metrics_plot.png')
    plt.close()
    logger.info("Metrics plot saved as 'metrics_plot.png'.")
except Exception as e:
    logger.error(f"Error generating plot: {e}")
    raise